In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import gc
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras import backend as K
import datetime
from sklearn.preprocessing import LabelEncoder
import keras
import tensorflow as tf
config = tf.compat.v1.ConfigProto(device_count = {'GPU':1,'CPU':6})
sess = tf.compat.v1.Session(config=config)
keras.backend.set_session(sess)

Using TensorFlow backend.


In [5]:
!mkdir Data

mkdir: cannot create directory ‘Data’: File exists


In [0]:
!cp -r "/content/drive/My Drive/Colab Notebooks/ML Project/ashrae-energy-prediction" -d "/content/Data/"

In [0]:
DATA_PATH = 'Data/ashrae-energy-prediction/'

In [0]:
train_df = pd.read_csv(DATA_PATH + 'train.csv')

# Remove outliers
train_df = train_df [ train_df['building_id'] != 1099 ]
train_df = train_df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')
building_df = pd.read_csv(DATA_PATH + 'building_metadata.csv')
weather_df = pd.read_csv(DATA_PATH + 'weather_train.csv')

In [0]:
# Original code from https://www.kaggle.com/aitude/ashrae-missing-weather-data-handling by @aitude

def fill_weather_dataset(weather_df):
    
    # Find Missing Dates
    time_format = "%Y-%m-%d %H:%M:%S"
    start_date = datetime.datetime.strptime(weather_df['timestamp'].min(),time_format)
    end_date = datetime.datetime.strptime(weather_df['timestamp'].max(),time_format)
    total_hours = int(((end_date - start_date).total_seconds() + 3600) / 3600)
    hours_list = [(end_date - datetime.timedelta(hours=x)).strftime(time_format) for x in range(total_hours)]

    missing_hours = []
    for site_id in range(16):
        site_hours = np.array(weather_df[weather_df['site_id'] == site_id]['timestamp'])
        new_rows = pd.DataFrame(np.setdiff1d(hours_list,site_hours),columns=['timestamp'])
        new_rows['site_id'] = site_id
        weather_df = pd.concat([weather_df,new_rows])

        weather_df = weather_df.reset_index(drop=True)           

    # Add new Features
    weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
    weather_df["day"] = weather_df["datetime"].dt.day
    weather_df["week"] = weather_df["datetime"].dt.week
    weather_df["month"] = weather_df["datetime"].dt.month
    
    # Reset Index for Fast Update
    weather_df = weather_df.set_index(['site_id','day','month'])

    air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weather_df.update(air_temperature_filler,overwrite=False)

    # Step 1
    cloud_coverage_filler = weather_df.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    # Step 2
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])
    weather_df.update(cloud_coverage_filler,overwrite=False)

    due_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weather_df.update(due_temperature_filler,overwrite=False)

    # Step 1
    precip_depth_filler = weather_df.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    # Step 2
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

    weather_df.update(precip_depth_filler,overwrite=False)

    weather_df = weather_df.reset_index()
    weather_df = weather_df.drop(['datetime','day','week','month'],axis=1)
        
    return weather_df

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def features_engineering(df):
    
    # Sort by timestamp
    df.sort_values("timestamp")
    df.reset_index(drop=True)
    
    # Add more features
    df["timestamp"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    df["hour"] = df["timestamp"].dt.hour
    df["weekend"] = df["timestamp"].dt.weekday
    df['square_feet'] =  np.log1p(df['square_feet'])
    
    # Remove Unused Columns
    drop = ["timestamp","sea_level_pressure", "wind_direction", "wind_speed","year_built","floor_count"]
    df = df.drop(drop, axis=1)
    gc.collect()
    
    # Encode Categorical Data
    le = LabelEncoder()
    df["primary_use"] = le.fit_transform(df["primary_use"])
    
    return df

In [10]:
weather_df = fill_weather_dataset(weather_df)
train_df = reduce_mem_usage(train_df,use_float16=True)
building_df = reduce_mem_usage(building_df,use_float16=True)
weather_df = reduce_mem_usage(weather_df,use_float16=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


Memory usage of dataframe is 757.31 MB
Memory usage after optimization is: 322.24 MB
Decreased by 57.4%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.8%
Memory usage of dataframe is 9.65 MB
Memory usage after optimization is: 2.66 MB
Decreased by 72.5%


In [11]:
train_df = train_df.merge(building_df, left_on='building_id',right_on='building_id',how='left')
train_df = train_df.merge(weather_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
del weather_df
gc.collect()

0

In [0]:
train_df = features_engineering(train_df)

In [13]:
target = np.log1p(train_df["meter_reading"])
features = train_df.drop('meter_reading', axis = 1)
del train_df
gc.collect()

0

In [0]:
#divide training and test data
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest = train_test_split(features,target,test_size=0.30,random_state=42)

In [0]:
#Compute the RMSLE
def RMSLE(pred,act): 
    return np.sqrt(np.sum((np.log(pred+1)-np.log(act+1))**2)/len(act))

In [0]:
#Compute the RMSLE
def RMSLE_model(pred,act): 
    return K.sqrt(K.mean(K.square(pred - act),axis = 0))

In [17]:
Xtrain.shape

(13896695, 11)

In [18]:
model = Sequential()
model.add(Dense(128,input_shape = (11,),activation='relu', name='fc1'))
model.add(Dropout(0.2))
model.add(BatchNormalization(epsilon=0.001))
model.add(Dense(64,activation='relu', name='fc2'))
model.add(Dropout(0.2))
model.add(BatchNormalization(epsilon=0.001))
model.add(Dense(32,activation='relu', name='fc3'))
model.add(Dropout(0.2))
model.add(BatchNormalization(epsilon=0.001))
model.add(Dense(16,activation='relu', name='fc4'))
model.add(Dropout(0.2))
model.add(BatchNormalization(epsilon=0.001))
model.add(Dense(1,activation='linear', name='output'))

optimizer = Adam(lr=0.001)
model.compile(optimizer,loss = ['mean_squared_error'], metrics=[RMSLE_model])
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 128)               1536      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
fc2 (Dense)                  (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       


In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/ML Project')

In [0]:
model.fit(Xtrain,Ytrain,verbose=2, batch_size = 3000, epochs = 1000)
model.save("Model_1000.h5") 




Epoch 1/1000




 - 47s - loss: 3.7629 - RMSLE_model: 1.8614
Epoch 2/1000
 - 43s - loss: 2.5970 - RMSLE_model: 1.6112
Epoch 3/1000
 - 43s - loss: 2.5508 - RMSLE_model: 1.5968
Epoch 4/1000
 - 43s - loss: 2.5188 - RMSLE_model: 1.5868
Epoch 5/1000
 - 43s - loss: 2.4983 - RMSLE_model: 1.5803
Epoch 6/1000
 - 43s - loss: 2.4798 - RMSLE_model: 1.5744
Epoch 7/1000
 - 43s - loss: 2.4657 - RMSLE_model: 1.5699
Epoch 8/1000
 - 43s - loss: 2.4543 - RMSLE_model: 1.5663
Epoch 9/1000
 - 43s - loss: 2.4400 - RMSLE_model: 1.5618
Epoch 10/1000
 - 43s - loss: 2.4391 - RMSLE_model: 1.5614
Epoch 11/1000
 - 43s - loss: 2.4277 - RMSLE_model: 1.5578
Epoch 12/1000
 - 43s - loss: 2.4243 - RMSLE_model: 1.5567
Epoch 13/1000
 - 42s - loss: 2.4200 - RMSLE_model: 1.5553
Epoch 14/1000
 - 42s - loss: 2.4147 - RMSLE_model: 1.5536
Epoch 15/1000
 - 42s - loss: 2.4130 - RMSLE_model: 1.5531
Epoch 16/1000
 - 41s - loss: 2.4094 - RMSLE_model: 1.5519
Epoch 17/1000
 - 41s - loss: 2.4034 - RMSLE_model: 1.5500
Epoch 18/1000
 -

In [0]:
model.load_weights("Model_1000.h5") 

In [20]:
model.fit(Xtrain,Ytrain,verbose=2, batch_size = 3000, epochs = 1000)
model.save("Model_2000.h5") 




Epoch 1/1000
 - 44s - loss: 2.2581 - RMSLE_model: 1.5024
Epoch 2/1000
 - 43s - loss: 2.2581 - RMSLE_model: 1.5024
Epoch 3/1000
 - 43s - loss: 2.2579 - RMSLE_model: 1.5023
Epoch 4/1000
 - 43s - loss: 2.2584 - RMSLE_model: 1.5025
Epoch 5/1000
 - 43s - loss: 2.2576 - RMSLE_model: 1.5022
Epoch 6/1000
 - 43s - loss: 2.2575 - RMSLE_model: 1.5021
Epoch 7/1000
 - 43s - loss: 2.2576 - RMSLE_model: 1.5022
Epoch 8/1000
 - 43s - loss: 2.2581 - RMSLE_model: 1.5024
Epoch 9/1000
 - 43s - loss: 2.2585 - RMSLE_model: 1.5025
Epoch 10/1000
 - 42s - loss: 2.2585 - RMSLE_model: 1.5025
Epoch 11/1000
 - 43s - loss: 2.2574 - RMSLE_model: 1.5021
Epoch 12/1000
 - 43s - loss: 2.2587 - RMSLE_model: 1.5026
Epoch 13/1000
 - 42s - loss: 2.2585 - RMSLE_model: 1.5025
Epoch 14/1000
 - 42s - loss: 2.2570 - RMSLE_model: 1.5020
Epoch 15/1000
 - 42s - loss: 2.2581 - RMSLE_model: 1.5024
Epoch 16/1000
 - 43s - loss: 2.2585 - RMSLE_model: 1.5025
Epoch 17/1000
 - 42s - loss: 2.2586 - RMSLE_model: 1.5025
Epoch 18/1000
 - 43s

In [20]:
model.load_weights("Model_2000.h5") 

In [0]:
model.fit(Xtrain,Ytrain,verbose=2, batch_size = 3000, epochs = 1000)
model.save("Model_3000.h5") 




Epoch 1/1000
 - 50s - loss: 2.2479 - RMSLE_model: 1.4990
Epoch 2/1000
 - 48s - loss: 2.2479 - RMSLE_model: 1.4990
Epoch 3/1000
 - 48s - loss: 2.2483 - RMSLE_model: 1.4991
Epoch 4/1000
 - 47s - loss: 2.2485 - RMSLE_model: 1.4992
Epoch 5/1000
 - 47s - loss: 2.2477 - RMSLE_model: 1.4989
Epoch 6/1000
 - 47s - loss: 2.2473 - RMSLE_model: 1.4988
Epoch 7/1000
 - 47s - loss: 2.2474 - RMSLE_model: 1.4988
Epoch 8/1000
 - 47s - loss: 2.2475 - RMSLE_model: 1.4988
Epoch 9/1000
 - 47s - loss: 2.2474 - RMSLE_model: 1.4988
Epoch 10/1000
 - 47s - loss: 2.2469 - RMSLE_model: 1.4986
Epoch 11/1000
 - 47s - loss: 2.2484 - RMSLE_model: 1.4991
Epoch 12/1000
 - 47s - loss: 2.2478 - RMSLE_model: 1.4989
Epoch 13/1000
 - 47s - loss: 2.2478 - RMSLE_model: 1.4989
Epoch 14/1000
 - 47s - loss: 2.2481 - RMSLE_model: 1.4990
Epoch 15/1000
 - 47s - loss: 2.2474 - RMSLE_model: 1.4988
Epoch 16/1000
 - 47s - loss: 2.2484 - RMSLE_model: 1.4991
Epoch 17/1000
 - 47s - loss: 2.2487 - RMSLE_model: 1.4992
Epoch 18/1000
 - 47s

In [0]:
model.fit(Xtrain,Ytrain,verbose=2, batch_size = 3000, epochs = 1000)
model.save("Model_4000.h5") 

In [0]:
model.fit(Xtrain,Ytrain,verbose=2, batch_size = 3000, epochs = 1000)
model.save("Model_5000.h5") 

In [0]:
#Training Predictions
y_predictTrain = model.evaluate(Xtrain)

IndexError: list index out of range

In [0]:
#Testing Predictions
y_predictTest = model.evaluate(Xtest)

In [0]:
#Evaluating on training Data
res=RMSLE(y_predictTrain,Ytrain)
res

In [0]:
#Evaluating on testing Data
res = RMSLE(y_predictTest,Ytest)
res

In [0]:
building_df = pd.read_csv(DATA_PATH + 'building_metadata.csv')
building_df = reduce_mem_usage(building_df)

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 65.4%


In [0]:
test_df = pd.read_csv(DATA_PATH + 'test.csv')
row_ids = test_df["row_id"]
test_df.drop("row_id", axis=1, inplace=True)
test_df = reduce_mem_usage(test_df)

Memory usage of dataframe is 954.38 MB
Memory usage after optimization is: 199.59 MB
Decreased by 79.1%


In [0]:
#building_df = pd.read_csv(DATA_PATH + 'building_metadata.csv')
test_df = test_df.merge(building_df,left_on='building_id',right_on='building_id',how='left')
del building_df
gc.collect()

8

In [0]:
weather_df = pd.read_csv(DATA_PATH + 'weather_test.csv')
weather_df = fill_weather_dataset(weather_df)
weather_df = reduce_mem_usage(weather_df)

C:\Users\Atifa Sarwar\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Memory usage of dataframe is 19.25 MB
Memory usage after optimization is: 9.05 MB
Decreased by 53.0%


In [0]:
test_df = test_df.merge(weather_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
del weather_df
gc.collect()

0

In [0]:
test_df.columns

Index(['building_id', 'meter', 'timestamp', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed'],
      dtype='object')

In [0]:
test_df = features_engineering(test_df)

In [0]:
predictions = model.predict(test_df)

KeyboardInterrupt: 

In [0]:
del test_df, models
gc.collect()

0

In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/ML')

In [0]:
results_df = pd.DataFrame({"row_id": row_ids, "meter_reading": np.clip(results, 0, a_max=None)})
del row_ids,results
gc.collect()
results_df.to_csv("submission_NN_Epoch10.csv", index=False)

NameError: name 'results' is not defined